In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

DEBUG = True

In [2]:
# 기상데이터 전처리
weather_df = pd.read_csv("../data/raw/Weather_2304_2503.csv")
weather_df.columns = [
    "city_id",  # 지점
    "city",  # 지점명
    "year_month",  # 일시
    "temperature_max",  # 최고기온(°C)
    "temperature_min",  # 최저기온(°C)
    "pressure",  # 평균현지기압
    "humidity",  # 평균상대습도
    "rain",  # 월합강수량
    "wind",  # 평균풍속
    "cloud",  # 평균운량
    "sunshine_duration",  # 합계일조시간
    "sunshine_rate",  # 일조율
]
weather_df["city"] = weather_df["city"].apply(
    lambda x: x.rstrip("시군도") if x != "완도" else x
)
weather_df["temperature_range"] = (
    weather_df["temperature_max"] - weather_df["temperature_min"]
)
weather_df = weather_df[
    [
        "city",
        "year_month",
        "temperature_max",
        "temperature_min",
        "temperature_range",
        "pressure",
        "humidity",
        "rain",
        "wind",
        "sunshine_rate",
    ]
]

weather_df.to_csv(
    "../data/preprocess/전처리__기상데이터.csv", index=False, encoding="utf-8-sig"
)
weather_df

,city,year_month,temperature_max,temperature_min,temperature_range,pressure,humidity,rain,wind,sunshine_rate
0,속초,2023-04,26.9,4.0,22.9,1011.6,61.0,57.0,2.2,53.61
1,속초,2023-05,34.4,5.5,28.9,1010.9,67.0,81.0,1.9,52.66
2,속초,2023-06,34.3,13.3,21.0,1004.5,74.0,200.1,1.8,55.26
3,속초,2023-07,35.1,19.6,15.5,1004.6,79.0,108.2,1.5,40.95
4,속초,2023-08,36.6,19.1,17.5,1004.5,85.0,669.6,1.4,47.36
...,...,...,...,...,...,...,...,...,...,...
2322,북부산,2024-11,24.6,-0.2,24.8,1021.4,68.0,82.0,1.7,54.61
2323,북부산,2024-12,19.3,-4.8,24.1,1022.2,50.0,0.0,1.5,73.74
2324,북부산,2025-01,14.9,-10.8,25.7,1023.2,52.0,3.5,1.6,72.83
2325,북부산,2025-02,16.8,-8.5,25.3,1022.9,43.0,22.0,1.9,72.93


In [3]:
# 지역별위탁병원 이용인원 전처리
hospital_df = pd.read_csv(
    "../data/raw/한국보훈복지의료공단_연월 및 지역별 위탁병원 이용인원_20250430.csv"
)
hospital_df.columns = ["year_month", "city", "count"]

hospital_df[["city_part1", "city_part2"]] = hospital_df["city"].str.split(
    " ", expand=True
)


def process_city(row):
    if row["city_part1"] in [
        "서울시",
        "부산시",
        "대구시",
        "인천시",
        "대전시",
        "광주시",
        "울산시",
    ]:
        return row["city_part1"].rstrip("시")
    else:
        return row["city_part2"].rstrip("시군")


hospital_df["city_processed"] = hospital_df.apply(process_city, axis=1)
hospital_df = (
    hospital_df.groupby(["city_processed", "year_month"])["count"].sum().reset_index()
)
hospital_df.columns = ["city", "year_month", "count"]

# 파일저장
hospital_df = hospital_df.sort_values(["city", "year_month"]).reset_index(drop=True)
hospital_df.to_csv(
    "../data/preprocess/전처리__지역별위탁병원_이용인원.csv",
    index=False,
    encoding="utf-8-sig",
)
hospital_df

,city,year_month,count
0,가평,2023-03,53
1,가평,2023-04,46
2,가평,2023-05,68
3,가평,2023-06,44
4,가평,2023-07,67
...,...,...,...
4037,횡성,2024-12,95
4038,횡성,2025-01,74
4039,횡성,2025-02,81
4040,횡성,2025-03,87


In [4]:
# 기상데이터와 병원이용인원 데이터 JOIN
weather_df = pd.read_csv("../data/preprocess/전처리__기상데이터.csv")
hospital_df = pd.read_csv("../data/preprocess/전처리__지역별위탁병원_이용인원.csv")

merged_df = weather_df.merge(hospital_df, on=["city", "year_month"], how="left")
merged_df = merged_df.dropna(subset=["count"])

if DEBUG:
    print("전처리__기상데이터:", len(weather_df))
    print("전처리__지역별위탁병원_이용인원:", len(hospital_df))
    print("전처리__입력데이터:", len(merged_df))

# 수치형 컬럼들 정규화
numeric_columns = [
    "temperature_max",
    "temperature_min",
    "temperature_range",
    "pressure",
    "humidity",
    "rain",
    "wind",
    "sunshine_rate",
]

scaler = StandardScaler()
merged_df[numeric_columns] = scaler.fit_transform(merged_df[numeric_columns])

if DEBUG:
    print("정규화 완료된 컬럼들:")
    for col in numeric_columns:
        print(
            f"{col}: mean={merged_df[col].mean():.4f}, std={merged_df[col].std():.4f}"
        )

# 수치형 컬럼들의 결측치를 평균값으로 채우기
for col in numeric_columns + ["count"]:
    if merged_df[col].isnull().sum() > 0:
        mean_val = merged_df[col].mean()
        merged_df[col] = merged_df[col].fillna(mean_val)

merged_df.to_csv(
    "../data/preprocess/전처리__입력데이터.csv", index=False, encoding="utf-8-sig"
)
merged_df

전처리__기상데이터: 2327
전처리__지역별위탁병원_이용인원: 4042
전처리__입력데이터: 1981
정규화 완료된 컬럼들:
temperature_max: mean=-0.0000, std=1.0003
temperature_min: mean=0.0000, std=1.0003
temperature_range: mean=-0.0000, std=1.0003
pressure: mean=0.0000, std=1.0003
humidity: mean=0.0000, std=1.0003
rain: mean=-0.0000, std=1.0003
wind: mean=0.0000, std=1.0003
sunshine_rate: mean=0.0000, std=1.0003


,city,year_month,temperature_max,temperature_min,temperature_range,pressure,humidity,rain,wind,sunshine_rate,count
0,속초,2023-04,0.075111,0.053896,-0.007646,0.508256,-1.025538,-0.551738,0.576214,0.336697,12.0
1,속초,2023-05,1.076957,0.187513,1.065357,0.457625,-0.423486,-0.378217,0.180443,0.253403,251.0
2,속초,2023-06,1.063599,0.882318,-0.347431,-0.005289,0.278908,0.482878,0.048519,0.481366,183.0
3,속초,2023-07,1.170463,1.443506,-1.331017,0.001944,0.780618,-0.181561,-0.347252,-0.773305,184.0
4,속초,2023-08,1.370832,1.398967,-0.973349,-0.005289,1.382670,3.877374,-0.479176,-0.211290,272.0
...,...,...,...,...,...,...,...,...,...,...,...
2298,남해,2024-11,-0.552713,0.000450,-0.740865,0.833743,-0.222802,-0.003702,-0.215328,-0.098185,79.0
2299,남해,2024-12,-1.180537,-0.623093,-0.329547,0.913306,-1.426906,-0.963849,0.180443,1.506321,61.0
2300,남해,2025-01,-1.941940,-1.104112,-0.383197,0.978403,-1.226222,-0.901671,0.048519,1.522980,59.0
2301,남해,2025-02,-1.327474,-0.917049,0.063887,0.971170,-1.928616,-0.733211,0.444290,1.255562,78.0


In [5]:
#################################################################
# city별 year_month 개수(데이터 개수)별로 도시 목록 출력
print("=== City별 year_month 개수별 도시 목록 ===")
city_counts = merged_df.groupby("city")["year_month"].count()
count_to_cities = {}

for city, cnt in city_counts.items():
    count_to_cities.setdefault(cnt, []).append(city)

for cnt in sorted(count_to_cities.keys(), reverse=True):
    cities = sorted(count_to_cities[cnt])
    print(f"{cnt} : {', '.join(cities)}")
#################################################################

# 고창, 산청, 울릉 데이터 삭제
merged_df = merged_df[~merged_df["city"].isin(["고창", "산청", "울릉"])].reset_index(
    drop=True
)

# 23개 year_month 값을 가지는 도시 찾기
city_counts = merged_df.groupby("city")["year_month"].count()
cities_with_23 = city_counts[city_counts == 23].index.tolist()

# year_month 전체 목록(정렬)
all_year_months = sorted(merged_df["year_month"].unique())

# 수치형 컬럼
numeric_columns = [
    "temperature_max",
    "temperature_min",
    "temperature_range",
    "pressure",
    "humidity",
    "rain",
    "wind",
    "sunshine_rate",
    "count",
]

from collections import defaultdict

# 도시별로 결측 year_month 찾아서 앞뒤 평균으로 채우기
new_rows = []
for city in cities_with_23:
    city_df = merged_df[merged_df["city"] == city].copy()
    city_ym_set = set(city_df["year_month"])
    missing_ym = [ym for ym in all_year_months if ym not in city_ym_set]
    if not missing_ym:
        continue
    # city_df를 year_month 기준으로 정렬
    city_df = city_df.sort_values("year_month").reset_index(drop=True)
    city_df_indexed = city_df.set_index("year_month")
    for ym in missing_ym:
        # 앞뒤 year_month 찾기
        ym_list = list(city_df_indexed.index)
        insert_pos = 0
        for i, y in enumerate(ym_list):
            if y > ym:
                insert_pos = i
                break
        else:
            insert_pos = len(ym_list)
        # 앞뒤 year_month
        prev_ym = ym_list[insert_pos - 1] if insert_pos > 0 else None
        next_ym = ym_list[insert_pos] if insert_pos < len(ym_list) else None
        if prev_ym is None or next_ym is None:
            # 앞 또는 뒤가 없으면 건너뜀 (이론상 없음)
            continue
        prev_row = city_df_indexed.loc[prev_ym]
        next_row = city_df_indexed.loc[next_ym]
        new_row = {"city": city, "year_month": ym}
        for col in numeric_columns:
            new_row[col] = (prev_row[col] + next_row[col]) / 2
        new_rows.append(new_row)

# 새로 생성한 행 추가
if new_rows:
    merged_df = pd.concat([merged_df, pd.DataFrame(new_rows)], ignore_index=True)
    # city, year_month 기준으로 정렬
    merged_df = merged_df.sort_values(["city", "year_month"]).reset_index(drop=True)

#################################################################
# city별 year_month 개수(데이터 개수)별로 도시 목록 출력
print("=== City별 year_month 개수별 도시 목록 ===")
city_counts = merged_df.groupby("city")["year_month"].count()
count_to_cities = {}

for city, cnt in city_counts.items():
    count_to_cities.setdefault(cnt, []).append(city)

for cnt in sorted(count_to_cities.keys(), reverse=True):
    cities = sorted(count_to_cities[cnt])
    print(f"{cnt} : {', '.join(cities)}")
#################################################################

# merged_df를 csv로 저장
merged_df.to_csv("../data/preprocess/전처리__입력데이터_윈도우24.csv", index=False)

=== City별 year_month 개수별 도시 목록 ===
46 : 고창
24 : 강릉, 거제, 거창, 경주, 광주, 구미, 금산, 김해, 남해, 대구, 대전, 동두천, 동해, 문경, 밀양, 보령, 보은, 봉화, 부산, 부여, 상주, 서귀포, 서산, 서울, 세종, 속초, 수원, 안동, 양산, 양평, 영월, 영주, 영천, 울산, 울진, 원주, 의령, 의성, 이천, 인천, 정선, 제주, 제천, 진주, 창원, 천안, 철원, 청주, 춘천, 충주, 태백, 통영, 파주, 포항, 함양, 합천, 홍성, 홍천
23 : 강진, 고흥, 광양, 군산, 남원, 목포, 보성, 부안, 순창, 순천, 여수, 영광, 영덕, 완도, 인제, 임실, 장수, 장흥, 전주, 정읍, 청송, 해남
20 : 산청
17 : 울릉
=== City별 year_month 개수별 도시 목록 ===
24 : 강릉, 강진, 거제, 거창, 경주, 고흥, 광양, 광주, 구미, 군산, 금산, 김해, 남원, 남해, 대구, 대전, 동두천, 동해, 목포, 문경, 밀양, 보령, 보성, 보은, 봉화, 부산, 부안, 부여, 상주, 서귀포, 서산, 서울, 세종, 속초, 수원, 순창, 순천, 안동, 양산, 양평, 여수, 영광, 영덕, 영월, 영주, 영천, 완도, 울산, 울진, 원주, 의령, 의성, 이천, 인제, 인천, 임실, 장수, 장흥, 전주, 정선, 정읍, 제주, 제천, 진주, 창원, 천안, 철원, 청송, 청주, 춘천, 충주, 태백, 통영, 파주, 포항, 함양, 합천, 해남, 홍성, 홍천
